In [ ]:
memo ="""
Todos
- [x] Update requirements.txt
- [ ] Test anaconda

Questions

- [x] Can we install pywin32?
- [x] Where is our item name?
- [ ] What's the cutoff date? Just in the past? Older than x months? Older than today?
- [x] What's the subject line?
"""

In [ ]:
# Important: To get win32 working, we need to do the following steps
# Using pip:
# pip install pypiwin32
# Then open an administrator shell and do
# venv\Scripts\activate
# python venv\Scripts\pywin32_postinstall.py -install

import win32com.client as win32
import os
import psutil
import subprocess
import pandas as pd

from dataclasses import dataclass
from template import MAIL_SUBJECT, MAIL_BODY

In [ ]:
# Read input file
EXCEL_INPUT_FILE = "EC_SDS_TESTFILE.xlsx"
excel_workbook = pd.read_excel(EXCEL_INPUT_FILE)

def get_expired_dict(df, expiry_date=None):
    """Takes a dataframe and an optional expiry_date, and returns
    a dict of all series older than the given expiry_date.
    
    If no expiry_date is given, the current date will be used.
    """
    if not expiry_date:
        expiry_date = pd.Timestamp.today()
    
    try:
        expired = df[df.last_update < expiry_date]
        return expired.to_dict('index')
    except AttributeError as e:
        print("Please check your input file contains a 'last_update' column.", e)


In [ ]:
# Set environment variables
OUTLOOK_APP_PATH = '"C:/Program Files/Microsoft Office/root/Office16/OUTLOOK.EXE"'
OUTLOOK_APP_NAME = 'OUTLOOK.EXE'

# Other configuration parameters
# TODO: Insert actual sender
EMAIL_SENDER = "TODO@todo.com"

# Email dataclass
@dataclass
class Email:
    sendFrom: str
    sendTo: str
    subject: str
    body: str

# Item dataclass
@dataclass
class Item:
    modifiedDate: str
    itemNo: str
    itemName: str
    supplierName: str
    supplierEmail: str
        
    def createSubject(self) -> str:
        return f"""最新版SDSの御提供のお願い [#{self.itemNo}]"""
    
    def createMessage(self) -> str:
        return f"""お取引先様各位

いつもお世話になっております。

現在弊社システムの定期的なメンテナンスを行っており、最新版のSDSを集めております。
つきましては貴社より購入しております、下記製品の和文と英文の両方のSDSの御提供をお願い致します。

{self.itemName} [#{self.itemNo}]

なお、以前に頂戴したSDSが弊社のシステムにあり記載されている改訂日が下記の通りです。
{self.modifiedDate.date()}

改訂されていない場合はその旨お知らせいただければ幸いでございます。
お忙しいところ恐れ入りますが、ご協力のほど何卒よろしくお願い申し上げます。
"""

In [ ]:
def open_outlook(path: str) -> None:
    """ Tries to open Outlook App from given path
    """
    try:
        subprocess.Popen(OUTLOOK_APP_PATH)
    except FileNotFoundError as e:
        print(f"Error: File not found at {OUTLOOK_APP_PATH}")

def app_is_running(app_name: str) -> bool:
    """ Checks running processes to see if a given app is running.
    Returns:
        True -- if app is running else False
    """
    # Get names for all running processes
    running_processes = (psutil.Process(pid).name() for pid in psutil.pids())
        
    for process in running_processes:
        if app_name in process:
            return True

    return False

def create_draft(mail: Email) -> None:
    """Takes an Email dataclass and saves a draft email.
    """
    outlook = win32.Dispatch('outlook.application')
    draft = outlook.CreateItem(0)
    draft.SentOnBehalfOfName = EMAIL_SENDER
    draft.To = mail.sendTo
    draft.Subject = mail.subject
    draft.body = mail.body
    draft.save()

In [ ]:
# Make sure we've got an email client running
if not app_is_running(OUTLOOK_APP_NAME):
    print("App not running, opening now.")
    open_outlook(OUTLOOK_APP_PATH)
else:
    print(f"{OUTLOOK_APP_NAME} is already running.")

In [ ]:
expired_sds_dict = get_expired_dict(excel_workbook)


# Create items and emails for all expired SDS
outbox = []
for sds in expired_sds_dict.values():
    item = Item(
        sds.get('last_update'),
        sds.get('art'),
        sds.get('art_name'),
        sds.get('vendor_master.salutation'),
        sds.get('vendor_master.email'),
    )
    
    email = Email(
        sendFrom=EMAIL_SENDER,
        sendTo=item.supplierEmail,
        subject=item.createSubject(),
        body=item.createMessage(),
    )
    outbox.append(email)
    
for mail in outbox:
    create_draft(mail)